In [54]:
import pandas as pd
google = pd.read_excel('https://raw.githubusercontent.com/ywchiu/tibame_tm/master/data/google_comments.xlsx',index_col=0)

In [55]:
def convertscore(score):
    if score >= 4:
        return 'good'
    elif score == 3:
        return 'soso'
    else:
        return 'bad'

In [56]:
google['status'] = google['score'].map(lambda e : convertscore(e))

In [1]:
google

NameError: name 'google' is not defined

In [73]:
import jieba
corpus = []
tags = []
for idx, rec in google[google['status'].isin(['good', 'bad'])].iterrows():
    corpus.append(list(jieba.cut(rec.get('comment'))))
    tags.append(rec.get('status'))

In [105]:
from gensim.models import word2vec
size = 50  # 產生多少維度 
min_count = 5 # 要算至少出現多少次數的字詞
workers = 4 # 使用多少個core 計算, -1 使用所有的core 進行計算 
window = 10  # 上下文的區間
iter = 1000 # 神經網路訓練的迭代數
sample = 1e-5 # 取樣的數量
model = word2vec.Word2Vec(corpus, 
                          workers = workers,
                          sample = sample,
                          size = size,
                          min_count=min_count,
                          window = window,
                          iter = iter)

In [106]:
import numpy as np
vecs = []
y = []
size = 50
for tag, s in zip(tags, corpus):
    vec = np.zeros(size).reshape((1, size))
    cnt = 0
    for w in s:
        if w in model:
            vec += model.wv.get_vector(w)
            cnt += 1
    if cnt > 0:
        vecs.append(vec / cnt)
        y.append(tag)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  if __name__ == '__main__':


In [107]:
len(vecs)

440

In [108]:
X = np.concatenate(vecs, axis = 0)

In [109]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [110]:
from sklearn.svm import SVC
clf = SVC(kernel = 'linear')
clf.fit(train_X, train_y)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [111]:
pred_y = clf.predict(test_X)

In [112]:
from sklearn.metrics import accuracy_score,confusion_matrix
accuracy_score(test_y, pred_y)

0.875

In [113]:
print(clf.classes_)
confusion_matrix(test_y, pred_y)

['bad' 'good']


array([[36,  4],
       [ 7, 41]])